# Application of Amplitude Estimation to Finances: Computing Integrals

The reference for this notebook will be:

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In finance one of the most important tasks is computing the "fair" price of a **derivative contract** whose definition (following Investopedia) is the following:

*A derivative is a contract between two or more parties whose value is based on an agreed-upon underlying financial asset (like a security) or set of assets (like an index).*

Usually, the problem of computing the price of a **derivative contract** can be reduced to computing an expectation of a given input function $f(x)$ when $x$ follows a probability density $p(x)$. 

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

This integral can be approximated by the Riemann sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

This value can be computed using **AE** techniques. In this notebook, we are going to review the function **q_solve_integral** from the module *quantum_integration* of the package *finance* of the *QQuantLib* library(**QQuantLib/finance/quantum_integration.py**).

This **q_solve_integral** uses the **Encoding** class from **QQuantLib.DL.encoding_protocols** and the **AE** class from **QQuantLib/AE/ae_class** for loading an input probability distribution $p(x)$ and a function $f(x)$ and computing the before integral using the different **AE** algorithms implemented in the **QQuantLib**.

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus
myqlm_qpus = ["python", "c"]
# QLM qpus accessed using Qaptiva Access library
qlmass_qpus = ["qlmass_linalg", "qlmass_mps"]
# QLM qpus: Only in local Quantum Learning Machine
qlm_qpus = ["linalg", "mps"]

linalg_qpu = get_qpu(myqlm_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Defining the problem

The first thing we need to do, for transforming our expectation computation in an **amplitude estimation** problem, is to define the density probability $p(x)$ and the function to evaluate $f(x)$. We cannot work with continuous variables so we need to discretise $p(x)$ and $f(x)$. 

We are going to define the following toy problem:

* Domain: our $x$ will be a set of $2^{n}$ integers numbers.

$$x \in \{0, 1, 2, ..., 2^n-1\}$$

* $p(x)$: Over our domain, we are going to define a properly normalised density distribution in the form:

$$p(x)=\frac{x}{\sum_{i=0}^{2^{n}-1}i}$$

* $f(x)$: Over our domain, we are going to define the following properly normalised function:

$$f(x) = \frac{x}{2^n-1}$$


**BE AWARE**

Is **MANDATORY** that $p(x)$ and $f(x)$ are properly normalised. Following conditions **must be** satisfied:

* For $p(x)$ is mandatory that: $\sum_{i=0}^{2^{n}} p(x_i) = 1$
* For $f(x)$ is mandatory that: $f(x_i) \leq 1 \forall i$

In [ ]:
a = 0.0

b = np.pi / 4.0

#n will define the maximum numbers of our domain
n = 3
x = np.linspace(a, b, 2 ** n)

#function definition
f_x = np.sin(x)
#function normalisation
f_x_normalisation = np.max(np.abs(f_x)) + 1e-8
norm_f_x = f_x / f_x_normalisation

#probability definition
p_x = x
#probability normalisation
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation

#Desired Integral
riemman = np.sum(p_x * f_x)

In [ ]:
riemman

In [ ]:
#Testing normalised conditions!

print('p(x) condition: {}'.format(np.sum(norm_p_x) <= 1))
print('f(x) condition: {}'.format(np.max(np.abs(norm_f_x)) <= 1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x, norm_p_x, label='p(x)')
ax1.set_ylabel('p(x)', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x, norm_f_x, color='r', label = 'f(x)')
ax2.set_ylabel('f(x)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
fig.legend(['p(x)', 'f(x)'])

## 2. q_solve_integral function

The *q_solve_integral* function from **QQuantLib.finance.quantum_integration** needs as input a Python dictionary with different keys. The function needs the following classes:

* *AE* class from **QQuantLib.AE.ae_class** (see Notebook **08_AmplitudeEstimation_Class**)
* *Encoding* class from **QQuantLib.AE.ae_class** (see Notebook **09_DataEncodingClass**)

So all the keys for configuring these classes can be used as keys of the input dictionary for the *q_solve_integral* function. 

Additionally, some keys are mandatory:

* *array_function*: numpy array with the desired function for encoding (this is $f(x)$).
* *array_probability*: numpy array with the desired probability (this is $p(x)$). This can be None (in this case a uniform distribution will be used).
* *encoding*: integer for selecting the encoding procedure to use (see Notebook **09_DataEncodingClass**).
* *ae_type*: string for selecting the AE algorithm to use for solving integral (see Notebook **08_AmplitudeEstimation_Class**)

The following cell creates a base Python dictionary for given to the *q_solve_integral* function

The return of the *q_solve_integral* function will be:

* *ae_estimation*: pandas DataFrame with the desired integral computed using **AE** techniques. In this case, normalisation due to the different encodes are managed by the function and the desired integral is returned transparently. There are 3 columns:
    * *ae*: integral value
    * *ae_l*: lower bound of the integral value (only **IQAE** and **RQAE**)
    * *ae_u*: upper bound of the integral value (only **IQAE** and **RQAE**)
* *solver_ae*: object created from the *AE* class and properly configured.
* *encode_class* : object created from the *Encode* class and properly configured.

In [ ]:
#typical AE input dictionary
m_k = [i for i in range(18)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-7,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 14,
    #IQPEAE
    'cbits_number': 10,
    #IQAE & RQAQE
    'epsilon': 0.0001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2,
    #For encoding class
    "multiplexor": True
}

#Now we add the data for encoding and the typw of AE

ae_dict.update({
    "array_function":norm_f_x,
    "array_probability": norm_p_x,
})

In [ ]:
from QQuantLib.finance.quantum_integration import q_solve_integral

The idea of the *q_solve_integral* function is converting the desired integral computation in an amplitude estimation (**AE**) problem and solving it with the different available **AE** techniques in **QQuantLib**.

In a general, an **AE** problem is the following, given an oracle $\mathcal{O}$ in the form:

$$\mathcal{O}|0\rangle = |\Psi\rangle = \sin(\theta) |\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,\tag{1}$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ we want an estimation of $\sin^2(\theta)$ by making measurements of the $|\Psi_0\rangle$ state. 

In general **Phase Estimation** algorithms (like **CQPEAE** and **IQPEAE**) allow to compute the $\sin^2(\theta)$ straightforwardly, but **QFT** is needed and quantum circuits become very complex. **AE** algorithms try to take advantage of the corresponding oracle Grover-like operator, $\mathcal{G}$, that acts:

$$\mathcal{G}^{m_k}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$

If $m_k$ is selected in such a way than $(2m_k+1)\theta \sim \frac{\pi}{2}$ then the measurement probability of $|\Psi_0\rangle$ state is near 1. The main problem is that the optimal $m_k$ depends on $\theta$. **AE** algorithms create systematic strategies for selecting $m_k$ for obtaining best $\sin^2(\theta)$.


## 3. Encoding protocol 0

First, we are going to use the first encoding protocol (*encoding=0*) as explained in Notebook **09_DataEncodingClass**) and then we are going to test all the possible implemented **AE** algorithms.

In this encoding protocol the Riemann sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)$$

will be codified as:

$$|\Psi\rangle 
= \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n}\otimes|0\rangle +  \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}\otimes|1\rangle \tag{2}$$

So comparing $(2)$ with $(1)$ we can define:

$$\sin(\theta)|\Psi_{0}\rangle = \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n} \otimes |0\rangle $$

and 

$$\sin^2(\theta) = \sum_{i=0}^{2^{n}-1}|p(x_i)f(x_i)|$$

In this case the *q_solve_integral* function will return the desired Riemann sum: $\sum_{i=0}^{2^{n}-1}|p(x_i)f(x_i)|$

In [ ]:
ae_dict.update({"encoding" : 0})

### 3.1 MLAE

Now we are going to use **MLAE** for solving the complete problem

In [ ]:
ae_dict.update({"ae_type" : "MLAE"})

In [ ]:
%%time
mlae_solution, mlae_object = q_solve_integral(**ae_dict)

In [ ]:
mlae_solution

In [ ]:
mlae_rieman = mlae_solution*f_x_normalisation*p_x_normalisation
print("MLAE Riemann :{}. Riemann: {}".format(mlae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mlae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
#we can access to different properties of the configured AE class used for doing the computations
mlae_object.schedule_pdf

In [ ]:
c = mlae_object.oracle 
%qatdisplay c --svg

### 3.2 IQAE

Now we are going to use **IQAE** for solving the complete problem

In [ ]:
ae_dict.update({"ae_type" : "IQAE"})
ae_dict.update({"epsilon" : 0.001})

In [ ]:
%%time
iqae_solution, iqae_object = q_solve_integral(**ae_dict)

In [ ]:
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

### 3.3 RQAE

Now we are going to use **RQAE** for solving the complete problem. Encoding 0 is not compatible with this **AE**. Instead of an error a warning is provided and an empty pandas DataFrame and a None will be returned.

In [ ]:
ae_dict.update({"ae_type" : "RQAE"})
ae_dict.update({"epsilon" : 0.001})

In [ ]:
%%time
rqae_solution, rqae_object = q_solve_integral(**ae_dict)

In [ ]:
rqae_solution

In [ ]:
rqae_object

### 3.4 MCAE

We are going to use **MCAE** procedure for computing integrals.

In [ ]:
%%time
ae_dict.update({"ae_type" : "MCAE"})
ae_dict.update({"shots" : 100000})
mcae_solution, mcae_object = q_solve_integral(**ae_dict)

In [ ]:
mcae_rieman = mcae_solution*f_x_normalisation*p_x_normalisation
print("MCAE Riemann :{}. Riemann: {}".format(mcae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mcae_rieman['ae'].iloc[0] - riemman)))

### 3.4 CQPEAE & IQPEAE

The **CQPEAE** and **IQPEAE** algorithms can be used for solving integrals too. But they are very demanding computing requirements. The cells will be left as comments. If the user wants to use will need only to transform in code cells. 

These methods can be executed in a reasonable time when **Eviden Quantum Learning Machine** can be used. At **CESGA** a **QLM** is deployed and It can be used (if you have a CESGA account) using the **qlmass** library. 

CESGA users can obtain more information at the following link: 

https://cesga-docs.gitlab.io/qlm-user-guide/qlm.html

#### CQPEAE

#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[0])

In [ ]:
ae_dict.update({"ae_type" : "CQPEAE"})
ae_dict.update({"qpu" : linalg_qpu})
#When bigger more precision and more computing demand
ae_dict.update({"auxiliar_qbits_number" : 14})

In [ ]:
cqpeae_solution, cqpeae_object = q_solve_integral(**ae_dict)

In [ ]:
cqpeae_rieman = cqpeae_solution*f_x_normalisation*p_x_normalisation
print("CQPEAE Riemann :{}. Riemann: {}".format(cqpeae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(cqpeae_rieman['ae'].iloc[0] - riemman)))

#### IQPEAE

ae_dict.update({"ae_type" : "IQPEAE"})
#When bigger more precision and more computing demand
ae_dict.update({"cbits_number" : 10})

iqpeae_solution, iqpeae_object = q_solve_integral(**ae_dict)

iqpeae_rieman = iqpeae_solution*f_x_normalisation*p_x_normalisation
print("IQPEAE Riemann :{}. Riemann: {}".format(iqpeae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqpeae_rieman['ae'].iloc[0] - riemman)))

### 3.6 Encoding Protocol 0 issues

As explained in the Notebook **09_DataEncodingClass** this encoding procedure can only codified properly strictly positive functions $f(x)$. So if we try to use this encoding procedure with a non-strictly positive function the **returned integral WILL BE INCORRECT!!**

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus
myqlm_qpus = ["python", "c"]
# QLM qpus accessed using Qaptiva Access library
qlmass_qpus = ["qlmass_linalg", "qlmass_mps"]
# QLM qpus: Only in local Quantum Learning Machine
qlm_qpus = ["linalg", "mps"]

linalg_qpu = get_qpu(myqlm_qpus[1])

In [ ]:
#Non strict define positive f(x)

a = np.pi - np.pi / 4.0

b = np.pi + np.pi / 8.0

#n will define the maximum numbers of our domain
n = 5
x = np.linspace(a, b, 2 ** n)

#function definition
f_x = np.sin(x)
#function normalisation
f_x_normalisation = np.max(np.abs(f_x)) + 1e-8
norm_f_x = f_x / f_x_normalisation

#probability definition
p_x = x
#probability normalisation
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation

#Desired Integral
riemman = np.sum(p_x * f_x)

In [ ]:
riemman

In [ ]:
#Testing normalised conditions!

print('p(x) condition: {}'.format(np.sum(norm_p_x) <= 1))
print('f(x) condition: {}'.format(np.max(np.abs(norm_f_x)) <= 1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x, norm_p_x, label='p(x)')
ax1.set_ylabel('p(x)', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x, norm_f_x, color='r', label = 'f(x)')
ax2.set_ylabel('f(x)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
fig.legend(['p(x)', 'f(x)'])

In [ ]:
#New function will be encoded!!! Not strictly definite positive function
ae_dict.update({
    "array_function":norm_f_x,
    "array_probability": norm_p_x,
})
ae_dict.update({"qpu" : linalg_qpu})

In [ ]:
%%time
#We are going to select one AE algorithm
ae_dict.update({"ae_type" : "IQAE", 'epsilon' : 0.001})
iqae_solution, iqae_object = q_solve_integral(**ae_dict)

In [ ]:
#Now the integral WILL BE WRONG because encoding procedure
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

## 4. Encoding protocol 1

We are going to use the second encoding protocol (*encoding=1*) as explained in Notebook **09_DataEncodingClass**) and then we are going to test all the possible implemented **AE** algorithms.

As explained in the notebook *09_DataEncodingClass* the desired integral is codified following ($3)$


$$|\Psi\rangle = \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |0\rangle \otimes |0\rangle \otimes |0\rangle_n \; + \; ... \tag{3}$$

So comparing $(3)$ and $(1)$

$$|\Psi_{0}\rangle = |0\rangle \otimes |0\rangle \otimes |0\rangle_n $$

and 

$$\mathbf{P}_{|\Psi_{0}\rangle} = \sin^2(\theta) = \left| \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) \right|^2$$

Again the output will take care of the encoding normalisation and the pure Riemman sum will be returned.

This procedure allows functions that are not strictly positive definite (into the domain, of  course). 

The following cells will use the new non-strictly positive define function with the encoding 1 and the different **AE** algorithms

In [ ]:
ae_dict.update({"encoding" : 1})

### 4.1 MLAE

Now we are going to use **MLAE** to solve the complete problem

In [ ]:
m_k = [2**i for i in range(12)]
ae_dict.update({
    "ae_type" : "MLAE", 
    'schedule': [m_k, [100]*len(m_k)]
})

In [ ]:
%%time
mlae_solution, mlae_object = q_solve_integral(**ae_dict)

In [ ]:
mlae_rieman = mlae_solution*f_x_normalisation*p_x_normalisation
print("MLAE Riemann :{}. Riemann: {}".format(mlae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mlae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
mlae_object.schedule_pdf

### 4.2 IQAE

Now we are going to use **IQAE** to solve the complete problem

In [ ]:
ae_dict.update({"ae_type" : "IQAE"})
ae_dict.update({"epsilon" : 0.0001})

In [ ]:
%%time
iqae_solution, iqae_object = q_solve_integral(**ae_dict)

In [ ]:
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

### 4.3 RQAE

Now we are going to use **RQAE** for solving the complete problem. In the case of the **RQAE** algorithm the amplitude of the $|\Psi_{0}\rangle$ is computed instead of its probability (so it can be negative). Again all the normalisations and internal computations are doing by the *q_solve_integral* function so the desired integral is returned to the user!!

All the **RQAE** variations can be used with this encoding (**sRQAE**, **mRQAE**, **eRQAE**)

#### Original RQAE

In [ ]:
ae_dict.update({"ae_type" : "RQAE", 'epsilon': 0.0005})

In [ ]:
%%time
rqae_solution, rqae_object = q_solve_integral(**ae_dict)

In [ ]:
rqae_rieman = rqae_solution*f_x_normalisation*p_x_normalisation
print("RQAE Riemann :{}. Riemann: {}".format(rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(rqae_rieman['ae'].iloc[0] - riemman)))

#### mRQAE

In [ ]:
%%time
ae_dict.update({"ae_type" : "mRQAE", 'epsilon': 0.0005})
m_rqae_solution, m_rqae_object = q_solve_integral(**ae_dict)
m_rqae_rieman = m_rqae_solution*f_x_normalisation*p_x_normalisation
print("mRQAE Riemann :{}. Riemann: {}".format(m_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(m_rqae_rieman['ae'].iloc[0] - riemman)))

#### sRQAE (RQAE with shots)

In [ ]:
%%time
# Now shots should be provided
ae_dict.update({"ae_type" : "sRQAE", 'epsilon': 0.0005, 'shots': 500})
s_rqae_solution, s_rqae_object = q_solve_integral(**ae_dict)
s_rqae_rieman = s_rqae_solution*f_x_normalisation*p_x_normalisation
print("sRQAE Riemann :{}. Riemann: {}".format(s_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(s_rqae_rieman['ae'].iloc[0] - riemman)))

#### eRQAE

In [ ]:
%%time
# Now we need to configure the scheduling function
schedule = {
    "type": "linear_linear",
    "ratio_slope_k": 3.5,
    "ratio_slope_gamma": -2.5
}
ae_dict.update({"ae_type" : "eRQAE", 'epsilon': 0.0005, 'erqae_schedule': schedule})
e_rqae_solution, e_rqae_object = q_solve_integral(**ae_dict)
e_rqae_rieman = e_rqae_solution*f_x_normalisation*p_x_normalisation
print("eRQAE Riemann :{}. Riemann: {}".format(e_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(e_rqae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
# we can acces to the schedules:
print(e_rqae_object.solver_ae.schedule_gamma)
print(e_rqae_object.solver_ae.schedule_k)

### 4.4 MCAE
We are going to use MCAE

In [ ]:
%%time
ae_dict.update({"ae_type" : "MCAE"})
ae_dict.update({"shots" : 100000})
mcae_solution, mcae_object = q_solve_integral(**ae_dict)

In [ ]:
mcae_rieman = mcae_solution*f_x_normalisation*p_x_normalisation
print("MCAE Riemann :{}. Riemann: {}".format(mcae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mcae_rieman['ae'].iloc[0] - riemman)))

## 5. Encoding protocol 2

We are going to use the third encoding protocol (*encoding=2*) as explained in Notebook **09_DataEncodingClass**) and then we are going to test all the possible implemented **AE** algorithms. In this encoding the desired integral is codified in the following way:

$$|\Psi \rangle = \sum_{i=0}^{2^{n}-1} p(x_i) f(x_i) |0\rangle \otimes |0\rangle_{n} \; + \; ... \tag{4}$$

So comparing $(4)$ and $(1)$

$$|\Psi_{0}\rangle = |0\rangle \otimes |0\rangle_n $$

and 

$$\mathbf{P}_{|\Psi_{0}\rangle} = \sin^2(\theta) = \left| \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) \right|^2$$

Again the output will take care of the encoding normalisation and the pure Riemann sum will be returned.

This procedure allows functions that are not strictly positive definite (into the domain, of  course). 

The following cells will use the new non-strictly positive define function with the encoding 1 and the different **AE** algorithms

In [ ]:
ae_dict.update({"encoding" : 2})

### 5.1 MLAE

Now we are going to use **MLAE** to solve the complete problem

In [ ]:
m_k = [i for i in range(12)]
ae_dict.update({
    "ae_type" : "MLAE", 
    'schedule': [m_k, [100]*len(m_k)]
})

In [ ]:
%%time
mlae_solution, mlae_object = q_solve_integral(**ae_dict)

In [ ]:
mlae_rieman = mlae_solution*f_x_normalisation*p_x_normalisation
print("MLAE Riemann :{}. Riemann: {}".format(mlae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mlae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
mlae_object.schedule_pdf

### 5.2 IQAE

Now we are going to use **IQAE** to solve the complete problem

In [ ]:
ae_dict.update({"ae_type" : "IQAE"})
ae_dict.update({"epsilon" : 0.001})

In [ ]:
%%time
iqae_solution, iqae_object = q_solve_integral(**ae_dict)

In [ ]:
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

### 5.3 RQAE

Now we are going to use **RQAE** to solve the complete problem.  In the case of the **RQAE** algorithm the amplitude of the $|\Psi_{0}\rangle$ is computed instead of its probability (so it can be negative). Again all the normalisations and internal computations are done by the *q_solve_integral* function so the desired integral is returned to the user!!

All the **RQAE** variations can be used with this encoding (**sRQAE**, **mRQAE**, **eRQAE**)

In [ ]:
ae_dict.update({"ae_type" : "RQAE", 'epsilon': 0.001})

In [ ]:
%%time
rqae_solution, rqae_object = q_solve_integral(**ae_dict)

In [ ]:
rqae_rieman = rqae_solution*f_x_normalisation*p_x_normalisation
print("RQAE Riemann :{}. Riemann: {}".format(rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(rqae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
rqae_object.schedule_pdf

#### mRQAE

In [ ]:
%%time
ae_dict.update({"ae_type" : "mRQAE", 'epsilon': 0.001})
m_rqae_solution, m_rqae_object = q_solve_integral(**ae_dict)
m_rqae_rieman = m_rqae_solution*f_x_normalisation*p_x_normalisation
print("mRQAE Riemann :{}. Riemann: {}".format(m_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(m_rqae_rieman['ae'].iloc[0] - riemman)))

In [ ]:
m_rqae_object.schedule_pdf

#### sRQAE (RQAE with shots)

In [ ]:
%%time
# Now shots should be provided
ae_dict.update({"ae_type" : "sRQAE", 'epsilon': 0.001, 'shots': 500})
s_rqae_solution, s_rqae_object = q_solve_integral(**ae_dict)
s_rqae_rieman = s_rqae_solution*f_x_normalisation*p_x_normalisation
print("sRQAE Riemann :{}. Riemann: {}".format(s_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(s_rqae_rieman['ae'].iloc[0] - riemman)))

#### eRQAE

In [ ]:
%%time
# Now we need to configure the scheduling function
schedule = {
    "type": "linear_linear",
    "ratio_slope_k": 3.5,
    "ratio_slope_gamma": -2.5
}
ae_dict.update({"ae_type" : "eRQAE", 'epsilon': 0.001, 'erqae_schedule': schedule})
e_rqae_solution, e_rqae_object = q_solve_integral(**ae_dict)
e_rqae_rieman = e_rqae_solution*f_x_normalisation*p_x_normalisation
print("eRQAE Riemann :{}. Riemann: {}".format(e_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(e_rqae_rieman['ae'].iloc[0] - riemman)))

### 5.4 MCAE
We are going to use MCAE

In [ ]:
%%time
ae_dict.update({"ae_type" : "MCAE"})
ae_dict.update({"shots" : 100000})
mcae_solution, mcae_object = q_solve_integral(**ae_dict)

In [ ]:
mcae_rieman = mcae_solution*f_x_normalisation*p_x_normalisation
print("MCAE Riemann :{}. Riemann: {}".format(mcae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(mcae_rieman['ae'].iloc[0] - riemman)))

## 6. Issues of AE algorithms

In general, the **AE** algorithms compute the probability of the state $|\Psi_{0}\rangle$ (**MLAE** or **IQAE**) so only positive values will be provided as outputs. So if the integral to compute is negative then these algorithms will return an incorrect value and the integral returned by the *q_solve_integral* will be wrong too (this is independent of the encoding procedure used)!!! We can see this in the following cells where a negative Riemann sum will be loaded into the states 

In [ ]:
#Negative Riemann sum!!

a = np.pi

b = np.pi + np.pi / 4.0

#n will define the maximum numbers of our domain
n = 5
x = np.linspace(a, b, 2 ** n)

#function definition
f_x = np.sin(x)
#function normalisation
f_x_normalisation = np.max(np.abs(f_x)) + 1e-8
norm_f_x = f_x / f_x_normalisation

#probability definition
p_x = x
#probability normalisation
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation

#Desired Integral
riemman = np.sum(p_x * f_x)

In [ ]:
riemman

In [ ]:
#Testing normalised conditions!

print('p(x) condition: {}'.format(np.sum(norm_p_x) <= 1))
print('f(x) condition: {}'.format(np.max(np.abs(norm_f_x)) <= 1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x, norm_p_x, label='p(x)')
ax1.set_ylabel('p(x)', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x, norm_f_x, color='r', label = 'f(x)')
ax2.set_ylabel('f(x)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
fig.legend(['p(x)', 'f(x)'])

In [ ]:
#New function will be encoded!!! Not strictly definite positive function
ae_dict.update({
    "array_function":norm_f_x,
    "array_probability": norm_p_x,
})

### encoding 1

In [ ]:
#Riemann sum negative!! Integration fails!
ae_dict.update(
    {"encoding" : 1,
     "ae_type" : "IQAE"
    })
iqae_solution, iqae_object = q_solve_integral(**ae_dict)
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

### encoding 2

In [ ]:
#Riemann sum negative!!  Integration fails!
ae_dict.update(
    {"encoding" : 2,
     "ae_type" : "IQAE",
     'epsilon': 0.001
    })
iqae_solution, iqae_object = q_solve_integral(**ae_dict)
iqae_rieman = iqae_solution*f_x_normalisation*p_x_normalisation
print("IQAE Riemann :{}. Riemann: {}".format(iqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(iqae_rieman['ae'].iloc[0] - riemman)))

## 7. RQAE algorithm saves the day

As explained the **RQAE** algorithm computes the amplitude of the  $|\Psi_{0}\rangle$ instead of its probability so it can detect the sign of the codified integral (always using encodings 1 and 2).

All the variations of **RQAE** algorithm will work too (**mRQAE**, **sRQAE**, **eRQAE**)

### encoding 1

In [ ]:
%%time
#Riemman sum negative!! 
ae_dict.update(
    {"encoding" : 1,
     "ae_type" : "RQAE",
     'epsilon': 0.0005
    })
rqae_solution, iqae_object = q_solve_integral(**ae_dict)
rqae_rieman = rqae_solution*f_x_normalisation*p_x_normalisation
print("RQAE Riemann :{}. Riemann: {}".format(rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(rqae_rieman['ae'].iloc[0] - riemman)))

### encoding 2

In [ ]:
%%time
#Riemman sum negative!! 
ae_dict.update(
    {"encoding" : 2,
     "ae_type" : "RQAE",
     'epsilon': 0.001
    })
rqae_solution, iqae_object = q_solve_integral(**ae_dict)
rqae_rieman = rqae_solution*f_x_normalisation*p_x_normalisation
print("RQAE Riemann :{}. Riemann: {}".format(rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(rqae_rieman['ae'].iloc[0] - riemman)))

#### mRQAE

In [ ]:
%%time
#Riemman sum negative!! 
ae_dict.update(
    {"encoding" : 2,
     "ae_type" : "mRQAE",
     'epsilon': 0.001
    })
m_rqae_solution, mrqae_object = q_solve_integral(**ae_dict)
m_rqae_rieman = m_rqae_solution*f_x_normalisation*p_x_normalisation
print("mRQAE Riemann :{}. Riemann: {}".format(m_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(m_rqae_rieman['ae'].iloc[0] - riemman)))

#### sRQAE

In [ ]:
%%time
#Riemman sum negative!! 
ae_dict.update(
    {"encoding" : 2,
     "ae_type" : "sRQAE",
     'epsilon': 0.001,
     'shots': 425,
    })
s_rqae_solution, srqae_object = q_solve_integral(**ae_dict)
s_rqae_rieman = s_rqae_solution*f_x_normalisation*p_x_normalisation
print("sRQAE Riemann :{}. Riemann: {}".format(s_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(s_rqae_rieman['ae'].iloc[0] - riemman)))

#### eRQAE

In [ ]:
%%time
#Riemman sum negative!! 
schedule = {
    "type": "exp_exp",
    "ratio_slope_k": 3.5,
    "ratio_slope_gamma": -4
}
ae_dict.update(
    {"encoding" : 2,
     "ae_type" : "eRQAE",
     'epsilon': 0.001,
     'erqae_schedule': schedule,
    })
e_rqae_solution, e_rqae_object = q_solve_integral(**ae_dict)
e_rqae_rieman = e_rqae_solution*f_x_normalisation*p_x_normalisation
print("eRQAE Riemann :{}. Riemann: {}".format(e_rqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(np.abs(e_rqae_rieman['ae'].iloc[0] - riemman)))